In [1]:
import pandas as pd
import numpy as np
# import tensorflow as tf

In [1]:
import tensorflow as tf
cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64)

2025-03-27 15:06:28.407410: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-03-27 15:06:28.407436: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2025-03-27 15:06:28.407442: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
I0000 00:00:1743084388.407452  126200 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1743084388.407467  126200 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/5


2025-03-27 15:06:33.854256: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


  8/782 ━━━━━━━━━━━━━━━━━━━━ 3:17 255ms/step - accuracy: 0.0097 - loss: 7.1175

KeyboardInterrupt: 

In [2]:
# read data
# df_wide = pd.read_csv('data/market_data.csv', header=[0,1], index_col=0)
df_wide = pd.read_csv('data/trade_df.csv', header=[0,1], index_col=0)

# Convert the index to datetime (the index holds the dates)
df_wide.index = pd.to_datetime(df_wide.index)

# Transform wide format to long format by stacking the first level of the columns (i.e., tickers)
# After stacking, each row will correspond to a unique date and ticker.
df = df_wide.stack(level=0).reset_index()

# Rename the resulting columns to have a proper 'Date' and 'Ticker' columns.
df.rename(columns={'Timestamp': 'Date', 'level_1': 'Ticker'}, inplace=True)

/var/folders/yw/jsw5n53s1cb1s2q6tt0msrm00000gn/T/ipykernel_4841/3265799413.py:10: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df_wide.stack(level=0).reset_index()


In [3]:
# Data Cleaning and Sorting
# Remove any duplicate rows and sort by ticker and date.
df.drop_duplicates(inplace=True)
df.sort_values(['Ticker', 'Date'], inplace=True)

# drop rows with missing values # should be moved to cleaning at some point when we start to combine datasets
df.dropna(inplace=True) # happens if there was one data point the first day of a given ticker but not the rest of the values

In [4]:
# 3. Create Momentum Features # should be moved to feature engineering at some point when we start to combine datasets
momentum_periods = {'mom_1m': 21, 'mom_3m': 63, 'mom_6m': 126, 'mom_12m': 252}

# compute percentage change over each period for each ticker separately.
for feature_name, period in momentum_periods.items():
    df[feature_name] = df.groupby('Ticker')['Trade Close'].transform(lambda x: x.pct_change(periods=period))

# drop rows where momentum features are not defined (if desired).
df.dropna(subset=list(momentum_periods.keys()), inplace=True)


In [5]:
# define the target variable
# for testing we define the target as the next-day return.
df['target'] = df.groupby('Ticker')['Trade Close'].pct_change().shift(-1)
# drop na and reset index
df.dropna(subset=['target'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
display(df)

,Date,Ticker,Trade Close,Trade High,Trade Low,Trade Open,Trade Volume,mom_1m,mom_3m,mom_6m,mom_12m,target
0,2022-01-31,AAB.CO,40.20,42.20,39.00,39.60,29909.0,-0.060600,-0.739056,-0.828062,-0.990016,0.149254
1,2022-02-28,AAB.CO,46.20,49.80,40.00,40.60,33153.0,0.034997,-0.669740,-0.799980,-0.987730,0.021645
2,2022-03-31,AAB.CO,47.20,47.60,42.80,45.20,13648.0,-0.174546,-0.685174,-0.743253,-0.986528,0.292373
3,2022-04-30,AAB.CO,61.00,72.00,46.40,47.40,13841.0,0.087849,-0.605551,-0.553769,-0.980877,-0.172131
4,2022-05-31,AAB.CO,50.50,61.00,49.00,61.00,6813.0,-0.111099,-0.715759,-0.617386,-0.982442,-0.184158
...,...,...,...,...,...,...,...,...,...,...,...,...
3349,2024-10-31,VWS.CO,129.10,146.80,124.80,145.95,58406035.0,-0.350538,0.163482,1.703099,4.360208,-0.150271
3350,2024-11-30,VWS.CO,109.70,137.25,97.22,129.50,98507648.0,-0.453141,0.100301,0.873292,5.551982,-0.105925
3351,2024-12-31,VWS.CO,98.08,113.40,93.10,109.70,68279567.0,-0.507136,-0.077849,0.785220,4.705410,0.008768
3352,2025-01-31,VWS.CO,98.94,107.50,91.60,102.00,77332526.0,-0.469377,-0.095282,0.963875,4.022927,0.024864


In [ ]:
feature_cols = [col for col in df.columns if col not in ['Date','Ticker','target']]
df_norm = df.dropna(subset=feature_cols)


# Optionally, filter out outliers or normalize features:
# For stability, you can normalize inputs (e.g., z-score per feature)
X_values = df_norm[feature_cols].values.astype('float32')
# Simple normalization: subtract mean, divide std
X_mean, X_std = X_values.mean(axis=0), X_values.std(axis=0)
X_values = (X_values - X_mean) / (X_std + 1e-8)

y_values = df_norm['target'].values.astype('float32')

# train-test split:
# set data before 2020 as training data and data after 2020 as testing data
train_indices = df_norm['Date'].dt.year < 2020
X_train, y_train = X_values[train_indices], y_values[train_indices]
# Here we use all data for training for illustration.
X_train, y_train = X_values, y_values

# 4. Define the MLP model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # single output for regression (excess return)
])

# Add L2 regularization or dropout if needed:
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-4), input_shape=(X_train.shape[1],)),
#     tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-4)),
#     tf.keras.layers.Dense(1)
# ])

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-03-27 15:01:29.702871: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-03-27 15:01:29.702897: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2025-03-27 15:01:29.702901: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
I0000 00:00:1743084089.702911  121184 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1743084089.702926  121184 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physic

In [8]:
model.compile(optimizer="adam",
              loss='mse',metrics=["accuracy"])

In [9]:
# 5. Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=64)

Epoch 1/5


: 